In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

In [ ]:
final_df = pd.read_csv('/Users/sachita/Documents/Projects/Summer/Winter2018/firedata/lstm.csv', sep=',',)

timesteps = 12 #timesteps needed later

df_shape = final_df.shape #Shape of dataframe before conversion to LSTM format
final_cols = final_df.columns.values.tolist() #Titles of columns in dataframe finally
n_vars = len(final_cols) #Number of variables we have for prediction

In [ ]:
final_df.head()

In [ ]:
#ONE HOT ENCODING OF Y (for input to lstm)

x = final_df.iloc[:, :-1].values
y = final_df.iloc[:, -1].values
print('Check on current dimensions of x and y: ', x.shape, y.shape) #Just a check on the dimension

y = y.astype(int) #type casting for one hot encoding
total_classes = len(range(0, 25)) #num of actual classes

y_enc = to_categorical(y, num_classes=total_classes)
y_enc = y_enc.astype(int)

print('Check on final dimensions of x and y: ', x.shape, y_enc.shape) #Just a check on the dimension



In [ ]:
#TRAIN-TEST SPLIT 

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_enc, test_size = 0.2, random_state = 0)

print('Shapes of x_train, y_train, x_test and y_test, before reshaping: ', x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# reshape input to be 3D [samples, timesteps, features] as expected by LSTM
x_train = x_train.reshape((x_train.shape[0], 12, 20))
x_test = x_test.reshape((x_test.shape[0], 12, 20))
print('Shapes of x_train, y_train, x_test and y_test, after reshaping: ', x_train.shape, y_train.shape, x_test.shape, y_test.shape)




In [ ]:
#CUSTOM METRIC

import tensorflow as tf
import keras.backend as K

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
#NEURAL NETWORK - REWRITE IN TENSORFLOW

# design network
model = Sequential()
model.add(LSTM(n_vars, input_shape=(x_train.shape[1], x_train.shape[2]), kernel_initializer='glorot_uniform'))
model.add(Dropout(0.2))
model.add(Dense(total_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

print("Fitting begins.")
# fit network
history = model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1, shuffle=True)

# # plot history
# pyplot.plot(history.history['loss'], label='train')
# pyplot.plot(history.history['val_loss'], label='test')
# pyplot.legend()
# pyplot.show()

In [ ]:
model.metrics_names

In [ ]:
loss, acc = model.evaluate(x_test, y_test, batch_size=32)

In [ ]:
print("Loss: %.5f" % (loss))
print("Accuracy: %.5f" % (acc))